# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#346efe; border-radius: 100px 100px; text-align:center">Torch inference notebook</h1></span>

<br>
<h2 style = "font-size:16px" 

This is the inference notebook made for training with  https://www.kaggle.com/vladvdv/pytorch-train-notebook-arcface-gem-pooling/notebook  
    
Modifications for version 30:
* replaced supervized KNeighborsClassifier with unsupervized NearestNeighbors   
* corrected gridsearch for determining optim "new_individual" threhsold* (there are used the same training data as the ones the model was trained, for training the NearestNeighbors algorithm, and then the same validation data that the model was trained to predict on the NearestNeighbors algorithm.  
   
To do:
* Implement all folds model blending

Modifications for version 31:
* Change original dataset to fins dataset
* Change the prediction function (design general function for max, avg and custom blend)

In [3]:
import pickle
import os
import gc
import cv2
import math
import copy
import time
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import joblib
from tqdm import tqdm
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import albumentations as A
from albumentations.pytorch import ToTensorV2
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("../input/timm-pytorch-image-models")
import timm
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [4]:
CONFIG = {"seed": 21, # choose your lucky seed
          "img_size": 512, # training image size
          "model_name": "tf_efficientnet_b4_ns", # training model arhitecture
          "num_classes": 4325, # total individuals in training data
          "test_batch_size": 4, # choose acording to the training arhitecture and image size 
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), # gpu
          "test_mode":False, # selects just the first 200 samples from the test data, usefull for debuging purposes
          "n_fold":3,
          # ArcFace Hyperparameters
          "s": 30.0, 
          "m": 0.30,
          "ls_eps": 0.0,
          "easy_margin": False,
          "rotate_h": False,
          "public_blend": False
          }

In [5]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [9]:
def get_test_file_path(id):
    return f"{TEST_DIR}/{id}"

def get_train_file_path(id):
    return f"{TRAIN_DIR}/{id}"

TEST_DIR = '../input/happy-whales-clean/images/images'
TRAIN_DIR = '../input/happy-whales-clean/images/images'
weights_path = "../input/weights/Loss8.5566_epoch10_latest_v12.bin"

if CONFIG["test_mode"]==True:
    df_test = pd.read_csv("../input/happy-whales-clean/test_png_processed_wdup.csv")[:2000]
    df_train = pd.read_csv("../input/happy-whales-clean/final_train.csv")[:2000]
else:
    df_test = pd.read_csv("../input/happy-whales-clean/updated_test.csv")
    df_train = pd.read_csv("../input/happy-whales-clean/updated_train.csv")  



df_test['file_path'] = df_test['image'].apply(get_test_file_path)
df_train['file_path'] = df_train['image'].apply(get_train_file_path)

encoder = LabelEncoder()
df_train['individual_id'] = encoder.fit_transform(df_train['class_id'])

train_labels = np.array(df_train['individual_id'].values)
#split into train and valid like in the training notebook for validating NearestNeighbors approach 
trainFold = 0 # this model was trained on fold 0

skf = StratifiedKFold(n_splits=CONFIG['n_fold'])
for fold, ( _, val_) in enumerate(skf.split(X=df_train, y=train_labels)):
      df_train.loc[val_ , "kfold"] = fold
df_train_cnn = df_train[df_train.kfold != trainFold].reset_index(drop=True)
df_valid_cnn = df_train[df_train.kfold == trainFold].reset_index(drop=True)


#hardcode dummy label for input in ArcMargin forward function
df_test['dummy_labels'] = 0
df_train_cnn['dummy_labels'] = 0
df_valid_cnn['dummy_labels'] = 0



In [27]:
import glob
all_files = set()
for filesa in glob.glob("/kaggle/input/happy-whales-clean/images/images/*.jpg"):
    all_files.add(filesa.split("/")[-1])
df_train_cnn = df_train_cnn[df_train_cnn['image'].isin(all_files)]
df_train_cnn = df_train_cnn.reset_index(drop=True)

In [33]:
df_train_cnn

,class_id,image,file_path,individual_id,kfold,dummy_labels
0,4250,MtRbr1esJ4DVAqp03BmK9GoyCiY6O2aEvhluUWIx.jpg,../input/happy-whales-clean/images/images/MtRb...,4198,1.0,0
1,3161,4voJfTzCPgkR13uy5xAD8dLVG2YasOXlhFjrntIc.jpg,../input/happy-whales-clean/images/images/4voJ...,3132,1.0,0
2,4209,nbDiZ190tugGoPXNJdVhB2AQkTLqUFES7W4zcjMm.jpg,../input/happy-whales-clean/images/images/nbDi...,4157,2.0,0
3,1337,dNJ4Bt17OLzhFcaq5IPwr20XSWxVnUy6mQ9CfDYe.jpg,../input/happy-whales-clean/images/images/dNJ4...,1327,1.0,0
4,2930,u47MGVyptd8l3TfKzUsoHeP5WAqjhQBNS0XJagrc.jpg,../input/happy-whales-clean/images/images/u47M...,2903,1.0,0
...,...,...,...,...,...,...
30964,1712,REWq7UnPcSIBi3AT4t0xj8oeaspHNKVGv2XkMb5Y.jpg,../input/happy-whales-clean/images/images/REWq...,1700,2.0,0
30965,943,rqBISlWJyf8AYcOhgZ1dGRoFiDxPzaLC9ewVs523.jpg,../input/happy-whales-clean/images/images/rqBI...,936,2.0,0
30966,1297,Hw57dRhVmaf1iMolUrZg2vuCAEb9SWkOPnL83tpD.jpg,../input/happy-whales-clean/images/images/Hw57...,1287,2.0,0
30967,1137,5QMo1VJFd6PgeRtbSs4mf2jrXUlzLcq8GIhATKxY.jpg,../input/happy-whales-clean/images/images/5QMo...,1128,2.0,0


In [34]:
class HappyWhaleDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df['dummy_labels'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        if img is None:
            print("Input image is empty", img_path)
            img = np.zeros((100, 100, 3), np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
        return {
            'image': img,
            'label': torch.tensor(label, dtype=torch.long)
        }
        

In [35]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

In [36]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, 
                 m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=CONFIG['device'])
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) ------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [37]:
class HappyWhaleModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super(HappyWhaleModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.drop = nn.Dropout(p=0.2, inplace=False)
        self.fc = nn.Linear(in_features,2048)
        self.arc = ArcMarginProduct(2048, 
                           CONFIG["num_classes"],
                           s=CONFIG["s"], 
                           m=CONFIG["m"], 
                           easy_margin=CONFIG["ls_eps"], 
                           ls_eps=CONFIG["ls_eps"])
    def forward(self, images, labels):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        pooled_drop = self.drop(pooled_features)
        emb = self.fc(pooled_drop)
        output = self.arc(emb,labels)
        return output,emb

In [38]:
data_transforms = {
    "test": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
    
}

In [39]:
@torch.inference_mode()
def inference(model, dataloader, device):
    model.eval()  
    outputList=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.long)
        _, outputs = model(images,labels)
        outputList.extend(outputs.cpu().detach().numpy())
    return outputList

In [40]:
model = HappyWhaleModel(CONFIG['model_name'])
model.to(CONFIG['device']);
model.load_state_dict(torch.load(weights_path))
#predict first on train dataset to extract embeddings
train_dataset = HappyWhaleDataset(df_train_cnn, transforms=data_transforms["test"])
train_loader = DataLoader(train_dataset, batch_size=CONFIG['test_batch_size'], 
                          num_workers=4, shuffle=False, pin_memory=True)

valid_dataset = HappyWhaleDataset(df_valid_cnn, transforms=data_transforms["test"])
valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['test_batch_size'], 
                          num_workers=4, shuffle=False, pin_memory=True)

test_dataset = HappyWhaleDataset(df_test, transforms=data_transforms["test"])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'], 
                          num_workers=4, shuffle=False, pin_memory=True)

In [41]:
def map_per_image(label, predictions):
    """Computes the precision score of one image.

    Parameters
    ----------
    label : string
            The true label of the image
    predictions : list
            A list of predicted elements (order does matter, 5 predictions allowed per image)

    Returns
    -------
    score : double
    """    
    try:
        return 1 / (predictions[:5].index(label) + 1)
    except ValueError:
        return 0.0

def map_per_set(labels, predictions):
    """Computes the average over multiple images.

    Parameters
    ----------
    labels : list
             A list of the true labels. (Only one true label per images allowed!)
    predictions : list of list
             A list of predicted elements (order does matter, 5 predictions allowed per image)

    Returns
    -------
    score : double
    """
    return np.mean([map_per_image(l, p) for l,p in zip(labels, predictions)])

**We are training a NearestNeighbors algorithm on the data that had been use as trainset in the train kernel and test the results on the validation data from the original kernel** (https://www.kaggle.com/vladvdv/pytorch-train-notebook-arcface-gem-pooling/notebook  )

In [43]:
df_train_cnn_predictions = np.array(inference(model, train_loader, CONFIG['device']))
df_valid_cnn_predictions = np.array(inference(model, valid_loader, CONFIG['device']))
train_cnn_labels = np.array(df_train_cnn['individual_id'].values)
valid_cnn_labels = np.array(df_valid_cnn['individual_id'].values)


 52%|█████▏    | 4026/7743 [05:03<04:13, 14.64it/s]

Input image is empty ../input/happy-whales-clean/images/images/UXPn1iK24TajF9sLCQpGbJhE5Mex0BwDVZyfIqdN.jpg


 54%|█████▍    | 4212/7743 [05:17<03:54, 15.04it/s]

Input image is empty ../input/happy-whales-clean/images/images/Do8NgnvcP6htYxeATaXw4SpbBVHQu15rfksjCULm.jpg


100%|██████████| 4168/4168 [05:16<00:00, 13.17it/s]


In [44]:
df_train.to_csv("map.csv")

In [76]:
test_cnn_predictions.shape

(21380, 2048)

In [ ]:
test_cnn_predictions =  np.array(inference(model, test_loader, CONFIG['device']))

allTrainData = np.concatenate((df_train_cnn_predictions,df_valid_cnn_predictions))
allTrainingLabels = np.concatenate((train_cnn_labels,valid_cnn_labels)) 

In [57]:
def GetDuplicates(elem, lst):
    if elem in lst:
        counter = 0
        elem_pos = []
        for i in lst:
            if i == elem:
                elem_pos.append(counter)
            counter = counter + 1
        return(elem_pos)    
    
def GetSubmission(train_data_list,test_data_list, train_labels_list,neighbors=5,metric='cosine', new_individual_thres=0.6,blendType = "avg"):
    df = pd.read_csv(r"../input/happy-whales-clean/updated_test.csv")     
    dfResults =pd.DataFrame(columns=set(train_labels_list[0]), index = df['image'])
    
    for it in range(len(train_data_list)):
        train_data = train_data_list[it]
        test_data = test_data_list[it]
        train_labels = train_labels_list[it]
        
        neigh = NearestNeighbors(n_neighbors=neighbors,metric=metric)
        neigh.fit(train_data)
        distances,idxs = neigh.kneighbors(test_data, return_distance=True)
        conf = 1-distances

        preds=[]
        
        for i in range(len(idxs)):
            preds.append(train_labels[idxs[i]])
        
        #remove duplicate predictions
        allPredsCleared = []
        allConfCleared = []
        for x in range(len(preds)):
            indexesToRemove=[]    
            for i in range(len(preds[x])):
                duplList = GetDuplicates(preds[x][i],preds[x])
                if len(duplList)>1:
                    for elem in duplList[1:]:
                        indexesToRemove.append(elem)
                
                
            indexesToRemove=list(set(indexesToRemove))
            
            # print(indexesToRemove)
            
            predsCleared = [i for j, i in enumerate(preds[x]) if j not in indexesToRemove]
            confCleared  = [i for j, i in enumerate(conf[x]) if j not in indexesToRemove]    
            allPredsCleared.append(predsCleared)
            allConfCleared.append(confCleared)
    
    
        # if prediction is nan create new record, otherwise append to existing one
        for i in range(len(allPredsCleared)):
            for j in range(len(allPredsCleared[i])):
                if np.isnan(dfResults.loc[df.iloc[i]['image']][allPredsCleared[i][j]]).any()==True:
                    dfResults.loc[df.iloc[i]['image']][allPredsCleared[i][j]] = [allConfCleared[i][j]]
                else:
                    dfResults.loc[df.iloc[i]['image']][allPredsCleared[i][j]].append(conf[i][j])
    
    #get mean predictions
    dictList=[]
    for imageName in (dfResults.index):
        results= dfResults.loc[imageName]
        tempDict={}
        for k in range(len(results)):
            if np.isnan(results[k]).any()!=True:
                if(blendType=="avg"):
                    blendConf=np.mean(results[k]) 
                if(blendType=="max"):
                    blendConf=np.max(results[k])
                if(blendType=="custom1"):
                    occurance = len(results[k])
                    halfFoldsNr = round(math.ceil(len(train_data_list)/2))
                    blendConf = np.max(results[k]) + ((occurance - halfFoldsNr)/10)
                if(blendType=="custom2"):
                    occurance = len(results[k])
                    halfFoldsNr = round(math.ceil(len(train_data_list)/2))
                    blendConf = np.mean(results[k]) + ((occurance - halfFoldsNr)/10)
                tempDict[results.index[k]] = blendConf
        dictList.append(tempDict)


    # calculate where to insert "new_individual"
    best5SubsAll=[]
    best5ConfAll=[]
    for i in range(len(dictList)):
        best5Subs=sorted(dictList[i], key=dictList[i].get,reverse=True)[:5]
        if len(best5Subs)<5:
            elementsToAppend= 5-len(best5Subs)
            for j in range(elementsToAppend):
                best5Subs.append(best5Subs[0])
        best5Conf=[dictList[i][best5Subs[0]], dictList[i][best5Subs[1]], dictList[i][best5Subs[2]],dictList[i][best5Subs[3]],dictList[i][best5Subs[4]]]
        best5ConfAll.append(best5Conf)
        best5SubsAll.append(best5Subs)
    predictTopDecoded={}
    
    for i in range(len(best5SubsAll)):
        predictTop = best5SubsAll[i]
        topValues = best5ConfAll[i]
        if (usePublic_new_individ == False):
            if topValues[0] < new_individual_thres:
                
                tempList=['new_individual',predictTop[0],predictTop[1],predictTop[2],predictTop[3]]
                predictTopDecoded[df.iloc[i]['image']] = tempList  
                
            elif topValues[1] < new_individual_thres:
        
                tempList=[predictTop[0],'new_individual',predictTop[1],predictTop[2],predictTop[3]]
                predictTopDecoded[df.iloc[i]['image']] = tempList     
                
            elif topValues[2] < new_individual_thres:
        
                tempList=[predictTop[0],predictTop[1],'new_individual',predictTop[2],predictTop[3]]
                predictTopDecoded[df.iloc[i]['image']] = tempList   
                
            elif topValues[3] < new_individual_thres:
                
                tempList=[predictTop[0],predictTop[1],predictTop[2],'new_individual',predictTop[3]]        
                predictTopDecoded[df.iloc[i]['image']] = tempList  
                
            elif topValues[4] < new_individual_thres:
        
                tempList=[predictTop[0],predictTop[1],predictTop[2],predictTop[3],'new_individual']        
                predictTopDecoded[df.iloc[i]['image']] = tempList         
                
            else:
                predictTopDecoded[df.iloc[i]['image']] = predictTop  
              

    return predictTopDecoded

In [65]:
train_labels_list[0].shape

(47641,)

In [66]:
train_data_list = [allTrainData]
test_data_list =[test_cnn_predictions]
train_labels_list = [allTrainingLabels]
neighbors=5
metric='cosine'
new_individual_thres=0.65
blendType = "max"
df = pd.read_csv(r"../input/happy-whales-clean/updated_test.csv")     

import glob
# for filesa in glob.glob("/kaggle/input/happy-whales-clean/images/images/*.jpg"):
#     all_files.add(filesa.split("/")[-1])
# df = df[df['image'].isin(all_files)]
# df = df.reset_index(drop=True)

dfResults =pd.DataFrame(columns=set(train_labels_list[0]), index = df['image'])

for it in range(len(train_data_list)):
    train_data = train_data_list[it]
    test_data = test_data_list[it]
    train_labels = train_labels_list[it]

    neigh = NearestNeighbors(n_neighbors=neighbors,metric=metric)
    neigh.fit(train_data)
    distances,idxs = neigh.kneighbors(test_data, return_distance=True)
    conf = 1-distances

    preds=[]

    for i in range(len(idxs)):
        preds.append(train_labels[idxs[i]])

    #remove duplicate predictions
    allPredsCleared = []
    allConfCleared = []
    for x in range(len(preds)):
        indexesToRemove=[]    
        for i in range(len(preds[x])):
            duplList = GetDuplicates(preds[x][i],preds[x])
            if len(duplList)>1:
                for elem in duplList[1:]:
                    indexesToRemove.append(elem)


        indexesToRemove=list(set(indexesToRemove))

        # print(indexesToRemove)

        predsCleared = [i for j, i in enumerate(preds[x]) if j not in indexesToRemove]
        confCleared  = [i for j, i in enumerate(conf[x]) if j not in indexesToRemove]    
        allPredsCleared.append(predsCleared)
        allConfCleared.append(confCleared)


    # if prediction is nan create new record, otherwise append to existing one
    for i in range(len(allPredsCleared)):
        for j in range(len(allPredsCleared[i])):
            if np.isnan(dfResults.loc[df.iloc[i]['image']][allPredsCleared[i][j]]).any()==True:
                dfResults.loc[df.iloc[i]['image']][allPredsCleared[i][j]] = [allConfCleared[i][j]]
            else:
                dfResults.loc[df.iloc[i]['image']][allPredsCleared[i][j]].append(conf[i][j])


KeyError: 58

In [106]:
dfs = pd.DataFrame(allPredsCleared, columns=["colummn1",'colummn2','colummn3','colummn4','colummn5'])
dfs.head()

,colummn1,colummn2,colummn3,colummn4,colummn5
0,1100,2438.0,170.0,1145.0,528.0
1,628,167.0,NaN,NaN,NaN
2,307,NaN,NaN,NaN,NaN
3,534,1394.0,559.0,1395.0,NaN
4,782,1098.0,1670.0,1117.0,NaN


In [107]:
dfs["image"] = df['image']

In [109]:
dfs.to_csv("result.csv")

In [99]:
df['image'].to_list()

['mwiNS62kZXUEJARdf3rxCpI0VztLO1B7guKTDGMj.jpg',
 'Dvtu5QLrVFq0JxjlcgmK94n3SC68iAphHXoBfzYM.jpg',
 'mldYZ9OiXh5P0ERHNcKLp78SbMtfkBx2rwJeszoa.jpg',
 '73gEZNzIV9SLcf14wkrTYUoHJOMuDetsvihP0mAq.jpg',
 'j7wJR0Qfl9xEgGW2MomUTtk4IAhYF53zbCesqKLy.jpg',
 'Y4QMGB2zxwjbfaZOgrpCFPsHk6qShlAJ1e03tmXc.jpg',
 'ytBpEZRqXFGKfM1n4CAmYJOislg2cVo35IDNvP7r.jpg',
 'mz7in1cRuNWE4xYpDr6JTXMgft2wOodsVAG0y9Lj.jpg',
 'yew8CENTQaORJ1KZMr2xVbH5mpAkjFt4qufoDn03.jpg',
 'mE0D4UA6dpVQ7toeNgHPqJGnTLFcBOfyYj2hi3S9.jpg',
 'OFEihKxSYm4fN2VokWgCMp3JTnlvIAqe0sUa5ZRD.jpg',
 'TR5lVd9chEgjFkp3PN0Iew7SAXrZYmGsOBDfMbHK.jpg',
 '7wfRpZq5aJV41dnPbE6QyGxAY0LmvWDFt28sIkTC.jpg',
 '9ubjKRtfeW4OkG6DhBzSmnP53yINwq2XQpsZC1rJ.jpg',
 'B9StPKboEa1u8ZXmVAfD6Tn2yNsQJczjqF5IO03x.jpg',
 'Ep6S0xM8QIqswd3FJKeTbBvX4ZVuiN5kG9t7HyA2.jpg',
 'zO5Fd2bfesmcNp3Sj8CTaQX9JAYEVuPnrRkBvWL1.jpg',
 's4l7dSrNufy1W0t6Y5hPZkpGDO9KXq3gUIJjAwHV.jpg',
 'wHlGJSem6xg2RD18rFNaVPcjkOsLZh9vKQWCzIXU.jpg',
 'pqLWnxu53gwIvHAEU6N2Rbrd08CB1DtjzhVXfZGc.jpg',
 'W1syljB3uz9PpqOTYx

In [83]:
allConfCleared

[[0.6045207, 0.5925591, 0.5641029, 0.5575105, 0.5568795],
 [0.58363, 0.49962366],
 [1.0],
 [0.81887245, 0.78916776, 0.787536, 0.7440249],
 [1.0, 1.0, 0.605379, 0.5734055],
 [0.52716166, 0.5223473, 0.5195427, 0.51457053],
 [0.8408599, 0.6102315],
 [0.7432345, 0.7118873],
 [0.9123347, 0.8713396],
 [0.6301823, 0.6143803, 0.60771775],
 [0.5994059, 0.5692923, 0.55842215, 0.5161427],
 [0.8051882, 0.5342725, 0.5170487, 0.46634597],
 [0.87803954],
 [0.9089638],
 [0.8747316],
 [0.99416745],
 [0.62773114, 0.54221684, 0.51205635, 0.50383687],
 [0.72062266, 0.6717094, 0.646449, 0.6265953, 0.6265208],
 [0.78774405, 0.55478436],
 [1.0, 1.0, 0.605379, 0.5734055],
 [0.91455275, 0.42691386],
 [0.63077295],
 [0.46022928, 0.45333344, 0.42481965, 0.421071, 0.42081618],
 [0.87742305, 0.8621004, 0.83186436],
 [0.96155643],
 [0.908012],
 [0.72139895, 0.6894332, 0.679784, 0.66855717, 0.66745293],
 [0.5145306, 0.5055825, 0.4473641, 0.444784],
 [0.8450895],
 [0.9872333, 0.9813771],
 [0.9981986, 0.7374759, 0.732

In [69]:

#get mean predictions
dictList=[]
print(results)
for imageName in (dfResults.index):
    results= dfResults.loc[imageName]
    tempDict={}
    for k in range(len(results)):
        print(results[k])
        if np.isnan(results[k]).any()!=True:
            if(blendType=="avg"):
                blendConf=np.mean(results[k]) 
            if(blendType=="max"):
                blendConf=np.max(results[k])
            if(blendType=="custom1"):
                occurance = len(results[k])
                halfFoldsNr = round(math.ceil(len(train_data_list)/2))
                blendConf = np.max(results[k]) + ((occurance - halfFoldsNr)/10)
            if(blendType=="custom2"):
                occurance = len(results[k])
                halfFoldsNr = round(math.ceil(len(train_data_list)/2))
                blendConf = np.mean(results[k]) + ((occurance - halfFoldsNr)/10)
            tempDict[results.index[k]] = blendConf
    dictList.append(tempDict)


# calculate where to insert "new_individual"
best5SubsAll=[]
best5ConfAll=[]
for i in range(len(dictList)):
    best5Subs=sorted(dictList[i], key=dictList[i].get,reverse=True)[:5]
    if len(best5Subs)<5:
        elementsToAppend= 5-len(best5Subs)
        for j in range(elementsToAppend):
            best5Subs.append(best5Subs[0])
    best5Conf=[dictList[i][best5Subs[0]], dictList[i][best5Subs[1]], dictList[i][best5Subs[2]],dictList[i][best5Subs[3]],dictList[i][best5Subs[4]]]
    best5ConfAll.append(best5Conf)
    best5SubsAll.append(best5Subs)
predictTopDecoded={}

for i in range(len(best5SubsAll)):
    predictTop = best5SubsAll[i]
    topValues = best5ConfAll[i]
    if (usePublic_new_individ == False):
        if topValues[0] < new_individual_thres:

            tempList=['new_individual',predictTop[0],predictTop[1],predictTop[2],predictTop[3]]
            predictTopDecoded[df.iloc[i]['image']] = tempList  

        elif topValues[1] < new_individual_thres:

            tempList=[predictTop[0],'new_individual',predictTop[1],predictTop[2],predictTop[3]]
            predictTopDecoded[df.iloc[i]['image']] = tempList     

        elif topValues[2] < new_individual_thres:

            tempList=[predictTop[0],predictTop[1],'new_individual',predictTop[2],predictTop[3]]
            predictTopDecoded[df.iloc[i]['image']] = tempList   

        elif topValues[3] < new_individual_thres:

            tempList=[predictTop[0],predictTop[1],predictTop[2],'new_individual',predictTop[3]]        
            predictTopDecoded[df.iloc[i]['image']] = tempList  

        elif topValues[4] < new_individual_thres:

            tempList=[predictTop[0],predictTop[1],predictTop[2],predictTop[3],'new_individual']        
            predictTopDecoded[df.iloc[i]['image']] = tempList         

        else:
            predictTopDecoded[df.iloc[i]['image']] = predictTop  

#         if (usePublic_new_individ == True):
#             if topValues[0] < new_individual_thres:
#                 
#                 tempList=['new_individual',predictTop[0],predictTop[1],predictTop[2],predictTop[3]]
#                 predictTopDecoded[df.iloc[i]['image']] = tempList             
#             else:
#                 tempList=[predictTop[0],'new_individual',predictTop[1],predictTop[2],predictTop[3]]
#                 predictTopDecoded[df.iloc[i]['image']] = tempList  

predictTopDecoded

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4338    NaN
4339    NaN
4340    NaN
4341    NaN
4342    NaN
Name: mwiNS62kZXUEJARdf3rxCpI0VztLO1B7guKTDGMj.jpg, Length: 4324, dtype: object
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


KeyError: 58

In [ ]:
from copy import deepcopy
predictions1 = deepcopy(predictions)
# for x in tqdm(predictions1):
#     predictions1[x] = ' '.join(str(predictions1[x]))

In [26]:
predictions_fin = pd.Series(predictions1).reset_index()
predictions_fin.columns = ['image','class_id']
predictions_fin.head()


,image,class_id
0,mwiNS62kZXUEJARdf3rxCpI0VztLO1B7guKTDGMj.jpg,"[4051, new_individual, 2647, 450, 3729]"
1,Dvtu5QLrVFq0JxjlcgmK94n3SC68iAphHXoBfzYM.jpg,"[new_individual, 1315, 462, 816, 823]"
2,mldYZ9OiXh5P0ERHNcKLp78SbMtfkBx2rwJeszoa.jpg,"[306, 2767, 1905, 2752, new_individual]"
3,tZaCF4XK2hPLTlW8YRdAnMD5mfSxzBujwVpUJ7b0.jpg,"[1287, new_individual, 1588, 705, 53]"
4,73gEZNzIV9SLcf14wkrTYUoHJOMuDetsvihP0mAq.jpg,"[558, 1391, 1392, 533, new_individual]"


In [27]:
predictions_fin.to_csv("beda.csv")